# Imports

In [1]:
#connect to gdrive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!ls "/content/drive/My Drive/Colab Notebooks"

 data		    data_reader.py	 Demo.ipynb   __pycache__
 data_preparer.py   data_vectorizer.py	'H&M.ipynb'   train.ipynb


In [23]:
#imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm
import sys
sys.path.insert(0,"/content/drive/My Drive/Colab Notebooks")
import data_preparer
from pathlib import Path
from keras.utils import to_categorical
import torch
import tensorflow as tf
import keras
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input, Dense, TimeDistributed
from keras.layers import LSTM, Bidirectional, Conv1D, concatenate, Permute, Dropout
from keras.layers import Dense, Flatten, Conv3D, MaxPooling3D, Dropout, BatchNormalization
from keras.callbacks import LearningRateScheduler

# Setup Training

In [4]:
# Setup traing data
vectors, letters = data_preparer.get_randomized_vectors(100)
length = data_preparer.get_max_frames()

/content/drive/My Drive/Colab Notebooks/data/sub001


In [5]:
items = set(letters)

items = {
    item: idx
    for idx, item in enumerate(items)
}

for idx, item in enumerate(letters):
    letters[idx] = items[item]

letters = np.array(letters)
letters = keras.utils.to_categorical(letters)

In [6]:
#vecotrs need to be setup differently because LSTM only taskes 3 dimesions ans cnn needs 4 dimensions
# vectors = data_preparer.make_numpy(vectors, length) # for LSTM model
vectors = data_preparer.make_numpy_cnn(vectors, length) #for CNN model

In [7]:
X_train, X_test, y_train, y_test = train_test_split(vectors, letters, test_size=0.2, random_state=13)

# Training

### **1. LSTM**

In [8]:
# # Mini Model
# model = keras.Sequential()
# model.add(keras.layers.LSTM(1))
# model.add(keras.layers.Dense(3, activation='sigmoid'))

In [9]:
# # Small Model
# model = keras.Sequential()
# model.add(keras.layers.LSTM(20))
# model.add(keras.layers.Dense(10, activation='sigmoid'))
# model.add(keras.layers.Dense(3, activation='sigmoid'))

In [10]:
# # Big Model
# model = keras.Sequential()
# model.add(keras.layers.LSTM(1860))
# model.add(keras.layers.Dense(100, activation='sigmoid'))
# model.add(keras.layers.Dense(10, activation='sigmoid'))
# model.add(keras.layers.Dense(3, activation='sigmoid'))

In [11]:
# model.compile(loss='mean_squared_error',
#               optimizer='Adamax',
#               metrics=['CategoricalAccuracy'])
# model.fit(X_train, y_train,
#           epochs=25,
#           verbose=1
#           )

In [12]:
# score, acc = model.evaluate(X_test, y_test)

In [13]:
#model = keras.models.load_model(r"models/small")

## **2. 3D-CNN**

In [17]:
#3D CNN model
model = Sequential()
model.add(Conv3D(20, kernel_size=(3, 3, 3), activation='relu', input_shape=(7, 20, length, 1)))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(10, kernel_size=(3, 3, 3), padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(10, kernel_size=(3, 3, 3), padding='same',activation='relu'))
model.add(BatchNormalization())


model.add(Flatten())
model.add(Dense(3, activation='sigmoid'))
# Compile the model
model.compile(loss='mean_squared_error',
              optimizer='Adamax',
              metrics=['CategoricalAccuracy'])

In [24]:
# Train the model -> takes very long on google colab
# Define the learning rate schedule function
def lr_schedule(epoch):
    lr = 0.001
    if epoch > 5:
        lr *= 0.1
    elif epoch > 10:
        lr *= 0.5
    return lr

# Create the learning rate scheduler callback
lr_scheduler = LearningRateScheduler(lr_schedule)

model.fit(X_train, y_train, epochs=10, verbose=1,batch_size=4, callbacks=[lr_scheduler])

Epoch 1/10
3600/3600 [==============================] - 526s 146ms/step - loss: 0.0920 - categorical_accuracy: 0.8181 - lr: 0.0010
Epoch 2/10
3600/3600 [==============================] - 519s 144ms/step - loss: 0.0597 - categorical_accuracy: 0.8865 - lr: 0.0010
Epoch 3/10
3600/3600 [==============================] - 518s 144ms/step - loss: 0.0490 - categorical_accuracy: 0.9062 - lr: 0.0010
Epoch 4/10
3600/3600 [==============================] - 518s 144ms/step - loss: 0.0410 - categorical_accuracy: 0.9212 - lr: 0.0010
Epoch 5/10
3600/3600 [==============================] - 521s 145ms/step - loss: 0.0357 - categorical_accuracy: 0.9326 - lr: 0.0010
Epoch 6/10
3600/3600 [==============================] - 524s 146ms/step - loss: 0.0324 - categorical_accuracy: 0.9381 - lr: 0.0010
Epoch 7/10
3600/3600 [==============================] - 520s 145ms/step - loss: 0.0297 - categorical_accuracy: 0.9435 - lr: 0.0010
Epoch 8/10
3600/3600 [==============================] - 529s 147ms/step - loss: 0.0

In [25]:
# Evaluate the model on the test set
score = model.evaluate(X_test, y_test, verbose=1)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

113/113 [==============================] - 29s 257ms/step - loss: 0.0327 - categorical_accuracy: 0.9344
Test loss: 0.03266969323158264
Test accuracy: 0.9344444274902344


In [29]:
model.save('/content/drive/My Drive/Colab Notebooks/cnn/')